# BPO

In [ ]:
from koselleck import *

In [ ]:
def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])

#     odf['num_lines']=odf.path.progress_apply(get_numlines)
#     odf['num_words']=odf['num_lines']*skipgram_n
    return odf.query('1680<=year<1970')

In [ ]:
dfskip=get_skipgrams()
dfskip

In [ ]:
def periodize(y,ybin=10):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [ ]:
periodize(1705)

In [ ]:
dfskip['period']=dfskip.year.apply(periodize)
dfskip[~dfskip.period.isnull()]

In [ ]:
dfskipruns=pd.concat([
    dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
    for i in range(25)
])
dfskipruns

In [ ]:
disable_gensim_logging()
# enable_gensim_logging()

In [ ]:
from gensim.models import FastText
from gensim.models import Word2Vec


def gen_and_save_model(dfskip,nskip=None,force=False,vector_size=100,window=10,min_count=10,epochs=5,workers=8):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame()

# model.wv.most_similar('value',topn=25)

In [ ]:
gby=['period','run']

In [ ]:
dfskipruns.groupby(gby).size()

In [29]:
dfskipruns_now = dfskipruns.query('period>="1870" & period<"1900" & run>"run_01"')
dfskipruns_now.groupby(gby).size()

period     run   
1870-1880  run_02    10
           run_03    10
           run_04    10
           run_05    10
           run_06    10
                     ..
1890-1900  run_21    10
           run_22    10
           run_23    10
           run_24    10
           run_25    10
Length: 72, dtype: int64

In [30]:
enable_gensim_logging()

In [ ]:
res=pmap_groups(
    gen_and_save_model,
    dfskipruns_now.groupby(gby),
    num_proc=3,
    kwargs=dict(force=True)
)

Mapping gen_and_save_model [x3]:   0%|          | 0/72 [00:00<?, ?it/s]collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 391589 words, keeping 22576 word types
collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #20000, processed 789179 words, keeping 33368 word types
collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 391589 words, keeping 22576 word types
PROGRESS: at sentence #30000, processed 1177435 words, keeping 40174 word types
PROGRESS: at sentence #10000, processed 391589 words, keeping 22576 word types
PROGRESS: at sentence #20000, processed 789179 words, keeping 33368 word types
PROGRESS: at sentence #40000, processed 1571009 words, keeping 46536 word types
PROGRESS: at sentence #20000, processed 789179 words, keeping 

In [ ]:
from gensim.models import KeyedVectors
m=KeyedVectors.load('/home/ryan/github/koselleck/data/models/bpo/1890-1900/run_01/model.bin')
m

In [ ]:
m.wv.most_similar('value')